# Authentication

Make an app and get keys form https://apps.twitter.co.

In [2]:
import requests
from requests_oauthlib import OAuth1

In [3]:
url = 'https://api.twitter.com/1.1/account/verify_credentials.json'
from donthackme import CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET
auth = OAuth1(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [4]:
verify_response = requests.get(url, auth=auth)

In [5]:
verify_response.json() # These are credentials connected to your account!

{u'contributors_enabled': False,
 u'created_at': u'Wed Jul 04 01:16:15 +0000 2012',
 u'default_profile': False,
 u'default_profile_image': False,
 u'description': u'Chief data scientist at iKnow Solutions, cook, climber, part-time nerd, recovering software developer, ex berliner, excited about lots of stuff.',
 u'entities': {u'description': {u'urls': []}},
 u'favourites_count': 9,
 u'follow_request_sent': False,
 u'followers_count': 39,
 u'following': False,
 u'friends_count': 87,
 u'geo_enabled': True,
 u'has_extended_profile': False,
 u'id': 626122134,
 u'id_str': u'626122134',
 u'is_translation_enabled': False,
 u'is_translator': False,
 u'lang': u'en',
 u'listed_count': 17,
 u'location': u'Oslo',
 u'name': u'Dirk Hesse',
 u'notifications': False,
 u'profile_background_color': u'000000',
 u'profile_background_image_url': u'http://abs.twimg.com/images/themes/theme1/bg.png',
 u'profile_background_image_url_https': u'https://abs.twimg.com/images/themes/theme1/bg.png',
 u'profile_backgr

# Search

Check the documentation at https://dev.twitter.com/rest/public/search.

In [6]:
search_url = 'https://api.twitter.com/1.1/search/tweets.json'
params = {'q': 'data science',
          'result_type': 'recent'}
search_response = requests.get(search_url, auth=auth, params=params)

In [ ]:
import json

In [ ]:
print json.dumps(search_response.json(), indent=2)

In [7]:
search_response.json().keys()

[u'search_metadata', u'statuses']

In [10]:
type(search_response.json()['statuses'])

list

In [8]:
search_response.json().get('statuses')[0].keys()

[u'contributors',
 u'truncated',
 u'text',
 u'is_quote_status',
 u'in_reply_to_status_id',
 u'id',
 u'favorite_count',
 u'entities',
 u'retweeted',
 u'coordinates',
 u'source',
 u'in_reply_to_screen_name',
 u'in_reply_to_user_id',
 u'retweet_count',
 u'id_str',
 u'favorited',
 u'retweeted_status',
 u'user',
 u'geo',
 u'in_reply_to_user_id_str',
 u'lang',
 u'created_at',
 u'in_reply_to_status_id_str',
 u'place',
 u'metadata']

In [11]:
search_response.json().get('statuses')[0]['entities'].keys()

[u'symbols', u'user_mentions', u'hashtags', u'urls']

In [16]:
print search_response.json().get('statuses')[0]['in_reply_to_user_id_str']

None


In [9]:
search_response.json().get('statuses')[0].get('user').keys()

[u'follow_request_sent',
 u'has_extended_profile',
 u'profile_use_background_image',
 u'default_profile_image',
 u'id',
 u'profile_background_image_url_https',
 u'verified',
 u'translator_type',
 u'profile_text_color',
 u'profile_image_url_https',
 u'profile_sidebar_fill_color',
 u'entities',
 u'followers_count',
 u'profile_sidebar_border_color',
 u'id_str',
 u'profile_background_color',
 u'listed_count',
 u'is_translation_enabled',
 u'utc_offset',
 u'statuses_count',
 u'description',
 u'friends_count',
 u'location',
 u'profile_link_color',
 u'profile_image_url',
 u'following',
 u'geo_enabled',
 u'profile_banner_url',
 u'profile_background_image_url',
 u'screen_name',
 u'lang',
 u'profile_background_tile',
 u'favourites_count',
 u'name',
 u'notifications',
 u'url',
 u'created_at',
 u'contributors_enabled',
 u'time_zone',
 u'protected',
 u'default_profile',
 u'is_translator']

In [ ]:
for i in search_response.json().get('statuses', []):
    print i.get('user').get('screen_name'), i.get('text')[:20]

In [ ]:
from collections import Counter
from urlparse import urlparse

In [ ]:
def get_urls(json_result):
    for status in json_result.get('statuses', []):
        for url in status['entities']['urls']:
             yield urlparse(url['expanded_url']).netloc

In [ ]:
list(get_urls(search_response.json()))

In [ ]:
cnt = Counter(get_urls(search_response.json()))
cnt

In [ ]:
cnt.most_common(5)

In [ ]:
json_response = search_response.json()

In [ ]:
next_path = json_response['search_metadata']['next_results']
print next_path

In [ ]:
more_tweets = requests.get(search_url + next_path, auth=auth).json()

In [ ]:
cnt.update(Counter(get_urls(more_tweets)))

In [ ]:
cnt.most_common(5)

# Pagination Strategies

In [ ]:
def tweets(base_url, params, auth):
    result = requests.get(base_url, params=params, auth=auth).json()
    for tweet in result.get('statuses', []):
        yield tweet
    print result.get('search_metadata', {})
    while result.get('search_metadata', {}).get('next_results', None):
        next_url = base_url + result['search_metadata']['next_results']
        print next_url
        result = requests.get(next_url, auth=auth).json()
        for tweet in result.get('statuses', []):
            yield tweet

In [ ]:
def urls(base_url, params, auth):
    for tweet in tweets(base_url, params, auth):
        for url in status['entities']['urls']:
            yield urlparse(url['expanded_url']).netloc

In [ ]:
from itertools import islice

In [ ]:
itercount = Counter(islice(urls(search_url, params, auth), 500))

In [ ]:
itercount.most_common()

In [ ]:
itercount.values()

## Streaming

In [ ]:
from itertools import islice

In [ ]:
r = requests.post('https://stream.twitter.com/1.1/statuses/filter.json',
                  params={'track': '#python'},
                  auth=auth,
                  stream=True)

In [ ]:
r

In [ ]:
print r.headers

In [ ]:
tweets = r.iter_lines()

In [ ]:
for tweet in islice(tweets, 20):
    try:
        tweet_dict = json.loads(tweet)
        print tweet_dict['text']
    except ValueError:
        print "Empty tweet for keep-alive"

In [ ]:
r.close()